In [55]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [56]:
import folium

from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap

In [65]:
import datetime

In [58]:
# ENTIRE 2019 CSV FILE

taxi = pd.read_csv('yellow_tripdata_2015-07.csv')
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-07-01 00:00:00,2015-07-01 00:15:26,1,3.5,-73.994156,40.751125,1,N,-73.976822,40.788567,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
1,1,2015-07-01 00:00:00,2015-07-01 00:22:22,1,3.9,-73.984657,40.768486,1,N,-74.000130,40.734898,2,17.0,0.5,0.5,0.00,0.0,0.3,18.30
2,1,2015-07-01 00:00:00,2015-07-01 00:07:42,1,2.3,-73.978889,40.762287,1,N,-74.004219,40.752533,2,9.0,0.5,0.5,0.00,0.0,0.3,10.30
3,1,2015-07-01 00:00:00,2015-07-01 00:39:37,1,9.2,-73.992790,40.742760,1,N,-73.971512,40.637154,1,33.0,0.5,0.5,8.55,0.0,0.3,42.85
4,1,2015-07-01 00:00:00,2015-07-01 00:05:34,1,1.1,-73.912430,40.769810,1,N,-73.920334,40.757442,1,6.0,0.5,0.5,2.00,0.0,0.3,9.30


In [59]:
total_rows=len(taxi.axes[0])
total_cols=len(taxi.axes[1])

print(total_rows)

11562783


In [60]:
new_taxi = taxi.drop(taxi.index[0:11557783])
new_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
11557783,2,2015-07-31 20:51:53,2015-07-31 20:56:29,1,1.41,-73.965797,40.762852,1,N,-73.951363,40.772839,1,6.5,0.5,0.5,2.34,0.0,0.3,10.14
11557784,2,2015-07-31 20:51:53,2015-07-31 21:24:05,1,4.72,-74.003738,40.720146,1,N,-73.958130,40.772953,2,22.0,0.5,0.5,0.00,0.0,0.3,23.30
11557785,2,2015-07-31 20:51:53,2015-07-31 21:08:24,1,2.34,-73.992851,40.758118,1,N,-73.985626,40.735493,1,12.5,0.5,0.5,2.76,0.0,0.3,16.56
11557786,2,2015-07-31 20:51:53,2015-07-31 21:20:58,1,5.09,-73.950272,40.779835,1,N,-74.005241,40.738182,2,21.0,0.5,0.5,0.00,0.0,0.3,22.30
11557787,1,2015-07-31 20:51:54,2015-07-31 21:00:32,1,1.20,-73.925529,40.770466,1,Y,-73.920044,40.776222,2,7.5,0.5,0.5,0.00,0.0,0.3,8.80


In [61]:
total_rows=len(new_taxi.axes[0])
total_cols=len(new_taxi.axes[1])

print(total_rows)

5000


In [62]:
# REDUCE CSV FILE TO COLUMNS NEEDED

taxi_reduced = new_taxi[['VendorID', 'tpep_pickup_datetime', 'pickup_longitude', 'pickup_latitude']]
taxi_reduced.head()

,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude
11557783,2,2015-07-31 20:51:53,-73.965797,40.762852
11557784,2,2015-07-31 20:51:53,-74.003738,40.720146
11557785,2,2015-07-31 20:51:53,-73.992851,40.758118
11557786,2,2015-07-31 20:51:53,-73.950272,40.779835
11557787,1,2015-07-31 20:51:54,-73.925529,40.770466


In [71]:
taxi_reduced['pickup_latitude'] = taxi_reduced['pickup_latitude'].apply(lambda x:round(x,3))
taxi_reduced['pickup_longitude'] = taxi_reduced['pickup_longitude'].apply(lambda x:round(x,3))

taxi_reduced.head()

/Users/michaelduval/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/michaelduval/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude
11557783,2,2015-07-31 20:51:53,-73.966,40.763
11557784,2,2015-07-31 20:51:53,-74.004,40.720
11557785,2,2015-07-31 20:51:53,-73.993,40.758
11557786,2,2015-07-31 20:51:53,-73.950,40.780
11557787,1,2015-07-31 20:51:54,-73.926,40.770


In [73]:
taxi_reduced['tpep_pickup_datetime'] = pd.to_datetime(taxi_reduced['tpep_pickup_datetime'])
taxi_reduced['tpep_pickup_datetime'] = taxi_reduced['tpep_pickup_datetime'].apply(lambda t: t.replace(second=0))
taxi_reduced

/Users/michaelduval/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/michaelduval/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude
11557783,2,2015-07-31 20:51:00,-73.966,40.763
11557784,2,2015-07-31 20:51:00,-74.004,40.720
11557785,2,2015-07-31 20:51:00,-73.993,40.758
11557786,2,2015-07-31 20:51:00,-73.950,40.780
11557787,1,2015-07-31 20:51:00,-73.926,40.770
...,...,...,...,...
11562778,1,2015-07-02 13:05:00,-73.992,40.750
11562779,1,2015-07-02 13:18:00,-73.991,40.750
11562780,1,2015-07-02 13:36:00,-73.992,40.750
11562781,1,2015-07-02 13:50:00,-73.992,40.749


In [74]:
taxi_reduced.groupby(['pickup_longitude','pickup_latitude'])['VendorID'].count()

pickup_longitude  pickup_latitude
-74.193           40.838               1
-74.035           40.727               1
-74.017           40.705               2
                  40.708               1
                  40.709               1
                                    ... 
-73.781           40.645               1
-73.777           40.645               7
                  40.646               2
-73.725           40.737               1
 0.000            0.000              762
Name: VendorID, Length: 1563, dtype: int64

In [77]:
taxi_good = taxi_reduced[(taxi_reduced != 0).all(1)]
taxi_good

,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude
11557783,2,2015-07-31 20:51:00,-73.966,40.763
11557784,2,2015-07-31 20:51:00,-74.004,40.720
11557785,2,2015-07-31 20:51:00,-73.993,40.758
11557786,2,2015-07-31 20:51:00,-73.950,40.780
11557787,1,2015-07-31 20:51:00,-73.926,40.770
...,...,...,...,...
11562778,1,2015-07-02 13:05:00,-73.992,40.750
11562779,1,2015-07-02 13:18:00,-73.991,40.750
11562780,1,2015-07-02 13:36:00,-73.992,40.750
11562781,1,2015-07-02 13:50:00,-73.992,40.749


In [78]:
taxi_good.dtypes

VendorID                         int64
tpep_pickup_datetime    datetime64[ns]
pickup_longitude               float64
pickup_latitude                float64
dtype: object

In [79]:
taxi_good['VendorID'] = taxi_good['VendorID'].astype(np.float)

/Users/michaelduval/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
pickup_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 10,)

'''
for index, row in taxi_good.iterrows():
    
    folium.CircleMarker([row['pickup_latitude'], row['pickup_longitude']],
                        radius=3,
                        
                        fill_color="#3db7e4", 
                        fill_opacity=0.9
                       ).add_to(pickup_map)
    count=count + 1


'''

hm_wide = HeatMap( list(zip(taxi_good.pickup_latitude.values, taxi_good.pickup_longitude.values, taxi_good.VendorID.values)),
                     min_opacity=0.2,
                     radius=5, blur=15,
                     max_zoom=1 
                 )
pickup_map.add_child(hm_wide)

pickup_map

In [83]:
# REFORMAT DATA

data = taxi_good.values
X = data[:, 1:4]
y = data[:, 3]

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [91]:
# ANY MISSING VALUES IN DATASET?

taxi_reduced[pd.isnull(taxi_reduced)].sum()

VendorID                0.0
tpep_pickup_datetime    0.0
pickup_longitude        0.0
pickup_latitude         0.0
dtype: float64

In [102]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [103]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=3))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [104]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [105]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 202       
Total params: 10,702
Trainable params: 10,702
Non-trainable params: 0
_________________________________________________________________


In [106]:
model.fit(
    X_train,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=2
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In [ ]:
TRAIN_SPLIT = 3500

In [ ]:
tf.random.set_seed(13)

In [ ]:
uni_data = taxi_reduced['pickup_longitude', 'pickup_latitude']
uni_data.index = taxi_reduced['tpep_pickup_datetime']
uni_data.head()

In [ ]:
uni_data.plot(subplots=True)

In [ ]:
uni_data = uni_data.values

In [ ]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

In [ ]:
uni_data = (uni_data-uni_train_mean)/uni_train_std

In [ ]:
univariate_past_history = 2000
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

In [ ]:
def create_time_steps(length):
  time_steps = []
  for i in range(-length, 0, 1):
    time_steps.append(i)
  return time_steps

In [ ]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')